In [7]:
import datetime
import gc
from pathlib import Path

import keras_tuner as kt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

del gpus

# Loading builders for datasets

In [8]:
RECIPES5K_DIR = "../Food Datasets/final-dataset/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "../Food Datasets/final-dataset/tfrecord/food101/1.1.0"
NUTRITION5K_DIR = "../Food Datasets/final-dataset/tfrecord/nutrition5k/1.0.0"


recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

In [9]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [10]:
nutrition5k_builder.info.splits["train"].num_examples

271407

In [11]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [12]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training category, ingredients and nutrients

In [13]:
# FULL DATASET
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
) = recipes5k_food101_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"]
)

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(
    nutrition5k_category_train,
    nutrition5k_category_val,
) = nutrition5k_builder.as_dataset(
    split=["train[:800]", "train[800:1000]"]
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)

In [14]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

Total training size : 85461
Total validation size : 21365


In [15]:
BATCH_SIZE = 256
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training ingredients and nutrients only

In [ ]:
# FULL DATASET
no_category_train, no_category_val = nutrition5k_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"], shuffle_files=True
)

In [ ]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")

In [ ]:
BATCH_SIZE = 256
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

# Model Development

In [17]:
class BaseModel:
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def _build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_units,
        overwrite_cached_model=True,
    ):
        raise NotImplementedError

    def build_and_compile(self, optimizer, **kwargs):
        # Evaluation metrics and loss
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                    average="macro",
                    name="macro_f1",
                ),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                    average="micro",
                    name="micro_f1",
                ),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                    average="macro",
                    name="macro_f1",
                ),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                    average="micro",
                    name="micro_f1",
                ),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        model = self._build(**kwargs)
        model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )
        self.model = model
        return model

    def freeze_category_classification_layers(self):
        assert self.model is not None
        for submodel in self.model.layers[1:]:
            if submodel.name == "category_output":
                submodel.trainable = False
                return

    def freeze_convolution_base(self):
        assert self.model is not None
        for submodel in self.model.layers[1:]:
            if submodel.name == "efficientnetB1":
                submodel.trainable = False
                return

    def get_callbacks(self):
        tensorboard_dir = f"./models/logs/{self.model_config_name}/{self.name}"
        checkpoint_dir = f"./temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_file = checkpoint_dir + "/{epoch:03d}-{val_loss:.4f}.hdf5"
        checkpoint_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_file, monitor="val_loss", save_best_only=True, mode="min"
        )
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=15)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_callback,
            reduce_lr_callback,
        ]

    ## Layers from bottom to top (classifier)  ##
    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
        return output_model

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    ## End of layers  ##

    def load_model(self, path):
        self.model = keras.models.load_model(path)
        return self.model

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        self.model.summary(expand_nested=True, show_trainable=True)

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert (
            self.model is not None
        ), "No model found. Please call build_and_compile() or load_model() before training."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        for submodel in self.model.layers[1:]:
            if submodel.name == "category_output":
                submodel.trainable = True
                return

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        for submodel in self.model.layers[1:]:
            if submodel.name == "efficientnetb1":
                print(len(submodel.layers))
                for layer in submodel.layers[fine_tune_at:]:
                    layer.trainable = True
                return

## Flat Model

In [18]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def _build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
        overwrite_cached_model=True,
    ):
        if not overwrite_cached_model:
            assert (
                self.model is not None
            ), "There is no previous model found. Do you mean to build a new model ?"
            return self.model
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

### MobileNetv3

In [ ]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_mobilenetv3",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_mobilenetv3.build_and_compile(
    keras.optimizers.Adam(),
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)


flat_mobilenetv3.print_summary()

In [ ]:
flat_mobilenetv3.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=10, verbose=1
)

### EfficientNetB1 (Best)

In [19]:
class FlatEfficientNetB1Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [20]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256-Full",
)

flat_efficientnet.build_and_compile(
    keras.optimizers.Adam(),
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)

flat_efficientnet.print_summary()

27033600/27018416 [==============================] - 0s 0us/step
Model: "augmentation_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 224, 224, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 224, 224, 3)       0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

Model: "preprocessing_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_____________________________

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=100, verbose=2
)

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/100


Cleanup called...
Cleanup called...


334/334 - 585s - loss: 3.8691 - category_output_loss: 2.2802 - ingredients_output_loss: 0.1602 - calorie_output_loss: 1.0365 - carbs_output_loss: 0.1414 - protein_output_loss: 0.1250 - fat_output_loss: 0.1257 - category_output_acc: 0.4446 - category_output_precision: 0.7464 - category_output_recall: 0.2924 - category_output_macro_f1: 0.4379 - category_output_micro_f1: 0.4446 - ingredients_output_precision: 0.0575 - ingredients_output_recall: 0.2593 - ingredients_output_macro_f1: 0.0019 - ingredients_output_micro_f1: 0.1471 - calorie_output_MAE: 1.0365 - carbs_output_MAE: 0.1414 - protein_output_MAE: 0.1250 - fat_output_MAE: 0.1257 - val_loss: 2.3973 - val_category_output_loss: 1.6364 - val_ingredients_output_loss: 0.0285 - val_calorie_output_loss: 0.5546 - val_carbs_output_loss: 0.0743 - val_protein_output_loss: 0.0313 - val_fat_output_loss: 0.0723 - val_category_output_acc: 0.5750 - val_category_output_precision: 0.8134 - val_category_output_recall: 0.4152 - val_category_output_macro_

Cleanup called...
Cleanup called...


...

334/334 - 549s - loss: 2.0046 - category_output_loss: 1.3553 - ingredients_output_loss: 0.0194 - calorie_output_loss: 0.4918 - carbs_output_loss: 0.0638 - protein_output_loss: 0.0213 - fat_output_loss: 0.0530 - category_output_acc: 0.6350 - category_output_precision: 0.8212 - category_output_recall: 0.5140 - category_output_macro_f1: 0.6310 - category_output_micro_f1: 0.6350 - ingredients_output_precision: 0.8092 - ingredients_output_recall: 0.4880 - ingredients_output_macro_f1: 0.0153 - ingredients_output_micro_f1: 0.1756 - calorie_output_MAE: 0.4918 - carbs_output_MAE: 0.0638 - protein_output_MAE: 0.0213 - fat_output_MAE: 0.0530 - val_loss: 2.0515 - val_category_output_loss: 1.4138 - val_ingredients_output_loss: 0.0188 - val_calorie_output_loss: 0.4810 - val_carbs_output_loss: 0.0619 - val_protein_output_loss: 0.0228 - val_fat_output_loss: 0.0532 - val_category_output_acc: 0.6355 - val_category_output_precision: 0.7905 - val_category_output_recall: 0.5481 - val_category_output_macro_

Cleanup called...
Cleanup called...


334/334 - 551s - loss: 1.9319 - category_output_loss: 1.2913 - ingredients_output_loss: 0.0191 - calorie_output_loss: 0.4856 - carbs_output_loss: 0.0630 - protein_output_loss: 0.0208 - fat_output_loss: 0.0521 - category_output_acc: 0.6497 - category_output_precision: 0.8273 - category_output_recall: 0.5310 - category_output_macro_f1: 0.6461 - category_output_micro_f1: 0.6497 - ingredients_output_precision: 0.8157 - ingredients_output_recall: 0.4984 - ingredients_output_macro_f1: 0.0156 - ingredients_output_micro_f1: 0.1765 - calorie_output_MAE: 0.4856 - carbs_output_MAE: 0.0630 - protein_output_MAE: 0.0208 - fat_output_MAE: 0.0521 - val_loss: 2.0498 - val_category_output_loss: 1.4093 - val_ingredients_output_loss: 0.0185 - val_calorie_output_loss: 0.4822 - val_carbs_output_loss: 0.0628 - val_protein_output_loss: 0.0224 - val_fat_output_loss: 0.0546 - val_category_output_acc: 0.6372 - val_category_output_precision: 0.7912 - val_category_output_recall: 0.5577 - val_category_output_macro_

Cleanup called...
Cleanup called...


334/334 - 549s - loss: 1.8741 - category_output_loss: 1.2396 - ingredients_output_loss: 0.0187 - calorie_output_loss: 0.4807 - carbs_output_loss: 0.0624 - protein_output_loss: 0.0207 - fat_output_loss: 0.0520 - category_output_acc: 0.6585 - category_output_precision: 0.8297 - category_output_recall: 0.5463 - category_output_macro_f1: 0.6550 - category_output_micro_f1: 0.6585 - ingredients_output_precision: 0.8195 - ingredients_output_recall: 0.5078 - ingredients_output_macro_f1: 0.0160 - ingredients_output_micro_f1: 0.1771 - calorie_output_MAE: 0.4807 - carbs_output_MAE: 0.0624 - protein_output_MAE: 0.0207 - fat_output_MAE: 0.0520 - val_loss: 2.0193 - val_category_output_loss: 1.3858 - val_ingredients_output_loss: 0.0183 - val_calorie_output_loss: 0.4772 - val_carbs_output_loss: 0.0615 - val_protein_output_loss: 0.0216 - val_fat_output_loss: 0.0550 - val_category_output_acc: 0.6418 - val_category_output_precision: 0.7876 - val_category_output_recall: 0.5631 - val_category_output_macro_

#### Fine tune the convolution base

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-4-4)-(1-8)-(1-32)-(1-4)]-(3-1024-512-256)-FULL-B256-(1e-5-1e-6)-FT",
)
flat_efficientnet.load_model("../input/foodnet/144-1.9698.hdf5")
flat_efficientnet.unfreeze_convolution_base(117)
flat_efficientnet.build_and_compile(
    keras.optimizers.Adam(learning_rate=1e-5),
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[32],
    independent_fat_units=[4],
    independent_carbs_units=[8],
    independent_calorie_units=[4, 4],
    overwrite_cached_model=False,
)

flat_efficientnet.print_summary()

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=100, verbose=2
)

#### Train on ingredients dataset only

### NASNet Mobile

In [ ]:
class FlatNASNetModel(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_nasnet",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.nasnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        nasnet_convolution_layers = keras.applications.nasnet.NASNetMobile(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        nasnet_convolution_layers.trainable = False
        return nasnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_nasnet = FlatNASNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_nasnet.build_and_compile(
    keras.optimizers.Adam(),
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)


flat_nasnet.print_summary()

In [ ]:
flat_nasnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=10, verbose=1
)

## Wide Slice Model

In [ ]:
class WideSliceModel(BaseModel):
    def _build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
        overwrite_cached_model=True,
    ):
        if not overwrite_cached_model:
            assert (
                self.model is not None
            ), "There is no previous model found. Do you mean to build a new model ?"
            return self.model
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        conv_layer = self.get_convolution_block()(prev_layer)
        wide_slice_layer = self.get_wideslice_conv_layers(prev_layer)(prev_layer)
        prev_layer = keras.layers.Concatenate()([conv_layer, wide_slice_layer])
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

    def get_wideslice_conv_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        rescale_layer = keras.layers.Rescaling(scale=1.0 / 255)(input_layer)
        wideslice_conv = keras.layers.Conv2D(filters=64, kernel_size=(5, 224))(
            rescale_layer
        )
        wideslice_conv = keras.layers.BatchNormalization()(wideslice_conv)
        wideslice_conv = keras.layers.ReLU()(wideslice_conv)
        wideslice_conv = keras.layers.GlobalAveragePooling2D()(wideslice_conv)
        return keras.Model(
            inputs=input_layer, outputs=wideslice_conv, name="wideslice_conv"
        )

### EfficientNetB1

In [ ]:
class WideSliceEfficientNetModel(WideSliceModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "wideslice_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="(64)-1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

wideslice_efficientnet.build_and_compile(
    keras.optimizers.Adam(),
    shared_units=[2048],
    independent_category_units=[512],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
    independent_ingredients_units=[1024, 512, 256],
)
wideslice_efficientnet.print_summary()

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset, epochs=100, verbose=2, validation_data=validation_dataset
)

# Testing

In [ ]:
import os
os.chdir(r'/kaggle/working')

!tar -czf temp.tar.gz ./temp

from IPython.display import FileLink

FileLink(r'temp.tar.gz')

In [1]:
import shutil
shutil.rmtree("./models")
shutil.rmtree("./temp")

In [ ]:
from pathlib import Path
Path("./models.tar.gz").unlink()
Path("./temp.tar.gz").unlink()

In [2]:
from pathlib import Path
Path("./temp").mkdir()
Path("./models").mkdir()  
Path("./temp/checkpoint").mkdir()

In [ ]:
import shutil
shutil.rmtree("./temp/backup")